# Network Setting 1

In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multiclass import OneVsRestClassifier
import tensorflow as tf

train = pd.read_csv('train_new_1.csv')
test = pd.read_csv('test_new.csv')

train = pd.read_csv('train_new_1.csv')
train = train.fillna(0)

test = pd.read_csv('test_new_1.csv')
test = test.fillna(0)

test_id = test['Id']
test = test.drop(columns = ['Id', 'text', 'reply_to_screen_name', 'Trump_flag', 'jfrketich_flag', 'hashtags', 'clean_tweet'], axis = 1)

train_lab = pd.get_dummies(train['sentiment_label'])
train = pd.concat([train.drop(columns = ['sentiment_label', 'sentiment'], axis = 1), train_lab], axis = 1)

test_lab = pd.get_dummies(test['sentiment_label'])
test = pd.concat([test.drop(columns = ['sentiment_label', 'sentiment'], axis = 1), test_lab], axis = 1)

## Defining input and target
X = train.drop(columns = ['text', 'reply_to_screen_name', 'Trump_flag', 'jfrketich_flag', 'hashtags', 'clean_tweet', 'country'], axis = 1)
Y = train['country']
Y = np.where(Y == 'us', 0, 
             np.where(Y == 'uk', 1, 
                      np.where(Y == 'canada', 2, 
                               np.where(Y == 'australia', 3,
                                        np.where(Y == 'ireland', 4, 5)))))

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.2)

## Scaling the data 
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
test = scaler.fit_transform(test)

# ## Defining model 
# model = tf.keras.models.Sequential([
#         tf.keras.layers.Dense(10, input_dim = 12, activation = 'relu'),
# #         tf.keras.layers.Dense(10, input_dim =  15, activation = 'relu'),
#         tf.keras.layers.Dense(6, activation = 'softmax')
# ])

# model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# ## Fitting model 
# model.fit(X_train, tf.keras.utils.to_categorical(Y_train, num_classes = 6), epochs = 20, batch_size = 32, validation_data = (X_test, tf.keras.utils.to_categorical(Y_test, num_classes = 6)))

# ## Predicting on test
# nn_pred = model.predict(test)
# nn_pred = np.argmax(nn_pred, axis = 1)

# ## Defining data to be exported
# data_out = pd.DataFrame({'Id': test_id, 'Category': nn_pred})
# data_out['Category'] = np.where(data_out['Category'] == 0, 'us',
#                                 np.where(data_out['Category'] == 1, 'uk',
#                                          np.where(data_out['Category'] == 2, 'canada',
#                                                   np.where(data_out['Category'] == 3, 'australia',
#                                                            np.where(data_out['Category'] == 4, 'ireland', 'new_zealand')))))
# data_out.to_csv('nn_submission_md7.csv', index = False)

In [17]:
X_train.shape

(48000, 6)

In [11]:
pd.get_dummies(train['sentiment_label'])

,negative,neutral,positive
0,0,0,1
1,0,1,0
2,0,0,1
3,0,0,1
4,0,1,0
...,...,...,...
239995,0,1,0
239996,0,0,1
239997,0,0,1
239998,1,0,0


In [3]:
nn_pred = model.predict(X_test)
nn_pred = np.argmax(nn_pred, axis = 1)

## Defining data to be exported
data_out = pd.DataFrame({'Actual': Y_test, 'Pred': nn_pred})
data_out['Actual'] = np.where(data_out['Actual'] == 0, 'us',
                              np.where(data_out['Actual'] == 1, 'uk',
                                       np.where(data_out['Actual'] == 2, 'canada',
                                                np.where(data_out['Actual'] == 3, 'australia',
                                                         np.where(data_out['Actual'] == 4, 'ireland', 'new_zealand')))))

data_out['Pred'] = np.where(data_out['Pred'] == 0, 'us',
                            np.where(data_out['Pred'] == 1, 'uk',
                                     np.where(data_out['Pred'] == 2, 'canada',
                                              np.where(data_out['Pred'] == 3, 'australia',
                                                       np.where(data_out['Pred'] == 4, 'ireland', 'new_zealand')))))
data_out.head()

,Actual,Pred
0,ireland,new_zealand
1,new_zealand,new_zealand
2,australia,uk
3,new_zealand,new_zealand
4,ireland,new_zealand


In [5]:
pd.crosstab(data_out['Actual'], data_out['Pred'])

Pred,canada,ireland,new_zealand,uk,us
Actual,,,,,
australia,0,3261,17467,5342,5950
canada,1,0,0,15029,16851
ireland,0,5066,23214,2136,1598
new_zealand,0,4208,24419,1658,1730
uk,0,0,0,15536,16471
us,0,0,0,14245,17818


In [6]:
data_out['Actual'].value_counts()

us             32063
australia      32020
new_zealand    32015
ireland        32014
uk             32007
canada         31881
Name: Actual, dtype: int64

In [4]:
from sklearn.metrics import classification_report

target_names = ['us', 'uk', 'canada', 'australia', 'ireland', 'new_zealand']
print(classification_report(data_out['Actual'], data_out['Pred'], target_names = target_names))

/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          us       0.00      0.00      0.00     32020
          uk       1.00      0.00      0.00     31881
      canada       0.40      0.16      0.23     32014
   australia       0.38      0.76      0.50     32015
     ireland       0.29      0.49      0.36     32007
 new_zealand       0.29      0.56      0.39     32063

    accuracy                           0.33    192000
   macro avg       0.39      0.33      0.25    192000
weighted avg       0.39      0.33      0.25    192000



/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
